In [1]:
import requests
import pandas as pd
from json.decoder import JSONDecodeError
import json

/Users/zhuxiaoxiao/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [88]:
api_list = [
'3bff9734cfbf4e1aa7b60c9fc224452d',
'ae48b5cb8ffe44c2b061a25785c74d97',
'65e16b9111744ab4840e59166dd659b9',
'4a8e398f23024576ae5e29c505e07270',
'81f3a64644484e32b542e9f48269b489',
'8a7893660af4483ba4da4b18349d6c79',
'9a49f2ddb5134355829ac5ae6946d718',
'e5b1e9fd2dee4f94aa30f7bfe73f21e4',
'0441e2cea6b84a5d976b4c0a037a9e60',
'7251ac941bb64e8cbee0eb58f00883a1',
'e672029cbbc944b586f436bd12a912d7']

In [118]:
companies = pd.read_csv('constituents.csv')
company = companies[['Security', 'Symbol']]

In [16]:
ticker = company['Symbol'].to_list()
ticker_dict = {}
for i in ticker:
    ticker_dict[i] = []

In [17]:
ticker2 = company['Security'].to_list()
ticker_dict2 = {}
for i in ticker2:
    ticker_dict2[i] = []

In [95]:
# Get company_news 
df = pd.DataFrame()
empty_list = []
base_url = "https://api.worldnewsapi.com/search-news"
counter = 0
#api_key = "3bff9734cfbf4e1aa7b60c9fc224452d"
for comp in company['Security'][351:]:
    try:
        api_key = api_list[counter]
        name = "ORG: " + comp
        params = {
        "language": "en",  
        "api-key": api_key, 
        # very few weekly data
        "earliest-publish-date":"2024-01-01",
        "latest-publish-date" : "2024-04-01",
        'text': name,
        "number":100
        }
        response = requests.get(base_url, params=params)
        data = response.json()
    except JSONDecodeError:
        print(f"took extremely long to get data for {comp}")
        continue
    try:
        df1 = pd.DataFrame(data['news'])
    except KeyError:
        try:
            if (data['code']) == 402:
                if counter == len(api_list) - 1:
                    print(comp)
                    print(api_key)
                    break
                else:
                    counter = counter + 1
                    api_key = api_list[counter]
                    print(f"update api key to {api_key}")
                    params = {
                    "language": "en",  
                    "api-key": api_key, 
                    # very few weekly data
                    "earliest-publish-date":"2024-01-01",
                    "latest-publish-date" : "2024-04-01",
                     'text': name,
                    "number":100
                    }
                    response = requests.get(base_url, params=params)
                    data = response.json()
            else:
                print(data)
        except KeyError:
            print("No news found")
            df1 = pd.DataFrame()

    try:
        ticker_dict2[comp] = df1['id'].to_list()
    except KeyError:
        print("No company data")
        empty_list.append(comp)
        ticker_dict2[comp] = []

    df = pd.concat([df, df1], ignore_index=True)

In [96]:
df

,id,title,text,url,image,publish_date,author,authors,language,source_country,sentiment
0,199090867,Easter egg hunts and activities,Get in the spring spirit with egg hunts for ki...,https://www.dailyherald.com/20240313/festivals...,https://imengine.public.prod.pdh.navigacloud.c...,2024-03-15 13:24:02,Posted March,['Posted March'],en,us,0.558
1,187896721,"Things to do Feb. 9-15: Peking Acrobats, Cocoa...",Ongoing “Downton Abbey — The Exhibition”: 10 a...,https://www.dailyherald.com/20240130/entertain...,https://imengine.public.prod.pdh.navigacloud.c...,2024-02-06 13:00:56,Luke Zurawski,['Luke Zurawski'],en,us,0.773
2,190942553,"Things to do Feb. 16-22: Chicago Auto Show, RV...",Ongoing “Downton Abbey — The Exhibition”: 10 a...,https://www.dailyherald.com/20240213/entertain...,https://imengine.public.prod.pdh.navigacloud.c...,2024-02-16 14:58:58,Luke Zurawski,['Luke Zurawski'],en,us,0.822
3,175801259,"The best exhibitions to see in London in 2024,...","There's no city better than London for art, an...",https://www.standard.co.uk/culture/exhibitions...,https://static.standard.co.uk/2023/12/13/14/28...,2024-01-02 07:25:52,"Graphic Thought Facility, Nancy Durrant","['Graphic Thought Facility', 'Nancy Durrant']",en,gb,0.708
4,175601473,More than 100 Bay Area festivals and fairs are...,From tasty food and drinks to awe-inspiring ar...,https://www.mercurynews.com/2024/01/01/more-th...,https://www.mercurynews.com/wp-content/uploads...,2024-01-01 15:30:56,Brittany Delay,['Brittany Delay'],en,us,1.000
...,...,...,...,...,...,...,...,...,...,...,...
36895,175582467,Arrests In Germany Over &#8216;Cologne Cathedr...,German police arrested three people on Sunday ...,https://independent.ng/arrests-in-germany-over...,https://independent.ng/wp-content/uploads/2018...,2024-01-01 14:00:02,Akin Yusuf,[Akin Yusuf],en,ng,-0.083
36896,175577659,"In the Peruvian Andes, the new year is welcome...","The fighting begins and ends with a hug, part ...",https://www.scmp.com/news/world/americas/artic...,https://cdn.i-scmp.com/sites/default/files/d8/...,2024-01-01 14:00:12,Agence France-Presse,[Agence France-Presse],en,cn,0.554
36897,175585095,Saskatchewan stories of the year: Experience R...,The troubled organization currently has a $17-...,https://leaderpost.com/news/local-news/saskatc...,https://smartcdn.gprod.postmedia.digital/leade...,2024-01-01 14:00:59,"Angela Amato, Kayle Neis","[Angela Amato, Kayle Neis]",en,ca,-0.116
36898,175577749,Every Premier League team's plan and top targe...,"As the calendar turns to 2024, the January tra...",https://www.irishmirror.ie/sport/soccer/premie...,https://i2-prod.irishmirror.ie/incoming/articl...,2024-01-01 14:01:51,Mirror Football,[Mirror Football],en,ie,-0.154


In [97]:
df.to_csv("news2.csv", index = False)

In [107]:
for (key1, value1), (key2, value2) in zip(ticker_dict2.items(), ticker_dict.items()):
    ticker_dict[key2] = value1

In [120]:
with open("tickers.json", "w") as json_file:
    json.dump(ticker_dict, json_file)

How to read the json:
with open("tickers.json", "r") as json_file:
    ticker_dict = json.load(json_file)